# Projet Data Visualisation
## Transformation des données pour leur utilisation

In [1]:
import pandas as pd
import unidecode as ud # pour enlever les accents aux libellés des colonnes
import re

### Lecture des données

In [2]:
files = ["releves_bancaires_2021.csv", "releves_bancaires_2020.csv"]
df_john_expenses = pd.DataFrame (columns = ["Libellé", "Date", "Débit euros", "Catégorie"])

for file in files:
    df = pd.read_csv(f"in/{file}", sep = ";")[["Libellé", "Date", "Débit euros", "Catégorie"]]
    df_john_expenses = pd.concat ([df_john_expenses, df])
df_john_expenses = df_john_expenses.loc[df_john_expenses["Débit euros"].notnull ()] # on ne garde que les débits

In [3]:
df_john_expenses.head()

,Libellé,Date,Débit euros,Catégorie
1,PRELEVEMENT \r\nBouygues Telecom,27/12/2021,11.99,Téléphone
2,PAIEMENT PAR CARTE \r\nLA FRANCAISE DES...,24/12/2021,8.5,Jeux
4,PAIEMENT PAR CARTE \r\nAMAZON EU SARL P...,23/12/2021,79.72,Cadeaux
5,PAIEMENT PAR CARTE \r\nFNAC LYON CEDEX 0 ...,20/12/2021,68.98,Cadeaux
6,PAIEMENT PAR CARTE \r\nGLOCAL DIFFUSION L...,20/12/2021,25.0,Cadeaux


In [4]:
df_expenses_by_population = pd.read_excel("in/raw_data.xlsx").rename(columns={"Unnamed: 0": "categorie", "Autres inactifs":"etudiants", "Ensemble":"francais"})[["categorie", "etudiants", "francais"]][:-1]

In [5]:
df_general_data = pd.DataFrame (data = [["Total (euros)", 10000., 16600.]], columns = ["categorie", "etudiants", "francais"]) # on ajoute les montants correspondants au total
df_expenses_by_population = pd.concat ([df_expenses_by_population, df_general_data])
df_expenses_by_population.head()

,categorie,etudiants,francais
0,Produits alimentaires et boissons non alcoolisées,16.768891,16.090613
1,Boissons alcoolisées et tabac,3.517955,2.875439
2,Articles d'habillement et chaussures,5.812929,5.012124
3,"Logement, eau, gaz, électricité et autres comb...",24.132486,16.251477
4,"Meubles, articles de ménage et entretien coura...",4.751010,5.684839


### Comparaison et mapping des catégories

In [6]:
df_john_expenses["Catégorie"].unique() # toutes les catégories dans df_john_expenses

array(['Téléphone', 'Jeux', 'Cadeaux', 'Restaurants', 'Transports',
       'Alimentation', 'Snacking', 'Cantine', 'Livres',
       'Mutuelle de santé', 'Loyer', 'Travaux', 'Pharmacie', 'Multimédia',
       'Evènements', 'Habillement', 'Autres dépenses', 'Sorties',
       'Frais bancaires', 'Ameublement/équipement', 'Santé/Bien être',
       "Retrait d'argent", 'Dons', 'Chaussures', 'Laverie',
       'Frais de scolarité', 'Assurance logement', 'Frais immobilier',
       'Remboursement'], dtype=object)

In [7]:
df_expenses_by_population["categorie"].unique() # toutes les catégories dans df_expenses_by_population

array(['Produits alimentaires et boissons non alcoolisées',
       'Boissons alcoolisées et tabac',
       "Articles d'habillement et chaussures",
       'Logement, eau, gaz, électricité et autres combustibles',
       'Meubles, articles de ménage et entretien courant de la maison',
       'Santé', 'Transports', 'Communications', 'Loisirs et culture',
       'Enseignement', 'Restauration et hôtels',
       'Biens et services divers (1)', 'Total (euros)'], dtype=object)

In [8]:
categories_mapping_operations = {
    "Habitation":["Habitation", "Travaux", "Electricité, gaz, eau", "Impôts : taxe foncière", "Impôts : taxe habitation",
                 "Habitation secondaire", "Loyer", "Jardin", "Assurance logement", "Ménage/repassage",
                 "Ameublement/équipement", "Frais immobilier", "Laverie"],
    "Loisirs":["Loisirs", "Multimédia", "Vacances", "Animaux", "Sport", "Sorties", "Evènements", "Jeux",
              "Hôtels", "Livres", "Restaurants"],
    "Communications":["Téléphone", "Internet/TV", "Communications"],
    "Alimentation" :["Alimentation", "Cantine", "Snacking"],
    "Transports":["Transports", "Véhicule", "Moto", "Péages", "Essence", "Location Voiture", "Assurance auto",
                 "Assurance moto", "Parking"],
    "Santé/Bien être":["Santé/Bien être", "Soin du corps", "Services à la personne", "Mutuelle de santé", "Pharmacie",
                      "Assurance santé", "Aides à domicile", "Coiffeur"],
    "Habillement":["Habillement", "Chaussures", "Accessoires"],
    "Autres":["Frais Pro", "Enfants", "Amendes", "Frais de justice", "Frais de scolarité", "Gardes d'enfant", "Dons", 
             "Impôts : autres", "Assurances : autres", "Autres dépenses",
             "Impôts : revenus", "Epargne", "Prêts", "Agios", "Frais bancaires", "Assurance vie", "Bourse", 
             "Retrait d'argent", "Prêt relais", "Cadeaux", "Remboursement"]
}
categories_mapping_operations = {subcat:cat for cat in categories_mapping_operations.keys() for subcat in categories_mapping_operations[cat]}

In [9]:
categories_mapping_expenses_cat = {
    "Produits alimentaires et boissons non alcoolisées": "Alimentation",
    "Boissons alcoolisées et tabac": "Alimentation",
    "Articles d'habillement et chaussures": "Habillement",
    "Logement, eau, gaz, électricité et autres combustibles": "Habitation",
    "Meubles, articles de ménage et entretien courant de la maison": "Habitation",
    "Santé": "Santé/Bien être",
    "Transports": "Transports",
    "Communications": "Communications",
    "Loisirs et culture": "Loisirs",
    "Enseignement": "Autres",
    "Restauration et hôtels": "Loisirs",
    "Biens et services divers (1)": "Autres",
    "Total (euros)": "Total (euros)"
}

### Transformation des données

In [10]:
df_john_expenses = df_john_expenses.rename(columns = {"Débit euros": "Montant"})
df_john_expenses = df_john_expenses.rename(columns = lambda col: ud.unidecode (col.lower ())) # noms des colonnes en minuscules et sans accents

In [11]:
df_john_expenses.head ()

,libelle,date,montant,categorie
1,PRELEVEMENT \r\nBouygues Telecom,27/12/2021,11.99,Téléphone
2,PAIEMENT PAR CARTE \r\nLA FRANCAISE DES...,24/12/2021,8.5,Jeux
4,PAIEMENT PAR CARTE \r\nAMAZON EU SARL P...,23/12/2021,79.72,Cadeaux
5,PAIEMENT PAR CARTE \r\nFNAC LYON CEDEX 0 ...,20/12/2021,68.98,Cadeaux
6,PAIEMENT PAR CARTE \r\nGLOCAL DIFFUSION L...,20/12/2021,25.0,Cadeaux


In [12]:
df_john_expenses["libelle"] = df_john_expenses["libelle"].map (lambda x: x.splitlines ()[1]) # supprimer "PAIEMENT PAR CARTE"
df_john_expenses["libelle"] = df_john_expenses["libelle"].map (lambda x: re.split ("\d+", x) [0]) # supprimer les numéros de compte, etc
df_john_expenses["libelle"] = df_john_expenses["libelle"].map (lambda x: re.sub ("\s{2,}", "", x)) # supprimer les espaces en trop

In [13]:
df_john_expenses["categorie"] = df_john_expenses["categorie"].map(categories_mapping_operations) # on oublie les sous-catégories et on ne garde que les catégories
df_john_expenses.head()

,libelle,date,montant,categorie
1,Bouygues Telecom,27/12/2021,11.99,Communications
2,LA FRANCAISE DES BOULOGNE,24/12/2021,8.5,Loisirs
4,AMAZON EU SARL PAYLI,23/12/2021,79.72,Autres
5,FNAC LYON CEDEX,20/12/2021,68.98,Autres
6,GLOCAL DIFFUSION LUNEL,20/12/2021,25.0,Autres


In [14]:
df_john_expenses["categorie"].value_counts() # nombre d'occurences

Alimentation       213
Habitation         104
Loisirs             96
Autres              85
Transports          59
Santé/Bien être     42
Communications      27
Habillement         15
Name: categorie, dtype: int64

In [15]:
df_expenses_by_population["categorie"] = df_expenses_by_population["categorie"].map(categories_mapping_expenses_cat) # on transforme les données de sorte à avoir les mêmes catégories que l'autre jeu de données
df_expenses_by_population = df_expenses_by_population.groupby("categorie").sum().reset_index() # on rassemble les mêmes catégories
df_expenses_by_population.head()

,categorie,etudiants,francais
0,Alimentation,20.286845,18.966052
1,Autres,14.800799,16.625232
2,Communications,3.649845,2.711766
3,Habillement,5.812929,5.012124
4,Habitation,28.883495,21.936316


### Sauvegarde des données

In [16]:
df_john_expenses.to_csv("out/john_expenses.csv", index=False)
df_expenses_by_population.to_csv("out/expenses_by_population.csv", index=False)